# Apply 메서드란?
 - 사용자가 작성한 함수를 한 번에 데이터프레임의 각 행과 열에 적용하여 실행할 수 있게 해 주는 메서드.
 - 즉, broadcasting해야 하는 경우에 apply 메서드를 적용함.
 - 대량 데이터 처리시 for문보다 빠르기 때문에 잘 알아두어야 함.

# 10-1. 간단한 함수 만들기

 - 함수의 기본적 형태.

        def my_function():
             #여기부터 코드 입력.

In [2]:
#제곱 함수 만들기.
def my_sq(x):
    return x ** 2

print(my_sq(4))
#n제곱 함수 만들기.
def my_exp(x, n):
    return x ** n

print(my_exp(2, 4))

16
16


# 10-2. apply 메서드 사용하기: 기초


 - 시리즈와 데이터프레임에 apply 메서드 사용하기.

In [10]:
import pandas as pd

df = pd.DataFrame({'a': [10, 20, 30], 'b': [20, 30, 40]})
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [11]:
#앞서 만든 my_exp를 사용하기 전, a열을 제곱하여 얻은 결과 보기.
print(df['a'] ** 2)

0    100
1    400
2    900
Name: a, dtype: int64


In [12]:
#apply 메서드를 적용한 결과값과 비교.

sq = df['a'].apply(my_sq) #apply 메서드에 함수 이름(my_sq)를 전달 ==> 시리즈의 모든 데이터에 제곱 함수를 적용!
print(sq)
#apply 메서드에 전달하는 함수(my_sq)가 1개의 인자를 받도록 구성되어있다면, 인자값을 생략해야 함.

0    100
1    400
2    900
Name: a, dtype: int64


In [13]:
#그렇다면, 2개의 인자를 전달해야 할 경우는?

ex = df['a'].apply(my_exp, n = 2) #apply 메서드의 첫 번째 인자에 함수 이름(my_exp), 두 번째 인자에는 함수의 두 번째 인자(n)를 전달.
print(ex)

ex2 = df['a'].apply(my_exp, n = 3) #apply 메서드의 첫 번째 인자에 함수 이름(my_exp), 두 번째 인자에는 함수의 두 번째 인자(n)를 전달.
print(ex2)

0    100
1    400
2    900
Name: a, dtype: int64
0     1000
1     8000
2    27000
Name: a, dtype: int64


 - 데이터프레임에 apply 적용하기.

In [14]:
df = pd.DataFrame({'a': [10, 20, 30], 'b': [20, 30, 40]})
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [17]:
#1개의 값을 전달받아 출력하는 함수.
def print_me(x):
    print(x)

In [19]:
#데이터프레임에 함수를 적용할 때에는, 함수를 적용할 방향을 column으로 할지 row로 할지 정해야 한다.
#column = 0, row = 1(default: 0)
print(df.apply(print_me, axis = 0))
print()
print(df['a'])
print()
print(df['b'])

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64
a    None
b    None
dtype: object

0    10
1    20
2    30
Name: a, dtype: int64

0    20
1    30
2    40
Name: b, dtype: int64


In [20]:
#3개의 인자를 입력받아 평균을 계산하는 함수
def avg_3(x, y, z):
    return (x+y+z) / 3

In [22]:
print(df.apply(avg_3))   
#이 경우, 3개의 인자가 필요한 avg_3함수에 열 단위 데이터(df['a'] or df['b'])가 전달됨.
#1개의 값만 avg_3함수에 전달되는 꼴이므로, 전달받은 인자값의 개수가 부족하다는 오류가 뜸.
#따라서 avg_3함수가 열 단위로 데이터를 처리할 수 있도록 수정해야 함.

TypeError: ("avg_3() missing 2 required positional arguments: 'y' and 'z'", 'occurred at index a')

In [24]:
def avg_3_apply(col):
    x = col[0]
    y = col[1]
    z = col[2]
    return (x+y+z) / 3

print(df.apply(avg_3_apply))

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64
a    20.0
b    30.0
dtype: float64


In [28]:
#위에서는 행 개수가 3개이란 것을 알고 있었지만, 일반적으로는 for문을 이용하여 아래와 같이 작성함.
def avg_3_apply(col):
    sum = 0
    for item in col:
        sum += item
    return sum / df.shape[0]     #df.shape = (3, 2) #df.shape[0] = 3(열 개수만큼 처리)

In [27]:
#avg_3_apply함수 응용: 행 방향으로 처리하는 함수 만들기.
def avg_2_apply(row):
    sum = 0
    for item in row:
        sum += item
    return sum / df.shape[1]     #df.shape = (3, 2) #df.shape[1] = 2(행 방향으로 처리)

print(df.apply(avg_2_apply, axis = 1))

3

# 10-3. apply 메서드 사용하기: 고급

 - titanic 데이터셋으로 apply 메서드 활용하기.

In [29]:
import seaborn as sns
titanic = sns.load_dataset("titanic")

print(titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB
None


In [32]:
#결측치 확인: count_missing함수(결측치 개수 반환)
import numpy as np
import pandas as pd

def count_missing(vec):
    null_vec = pd.isnull(vec)     #pandas의 isnull 메서드 활용, 누락값 유무에 따라 True/False를 적용한 데이터프레임 형성.
    null_count = np.sum(null_vec) #이 값을 numpy의 sum 메서드에 전달하여 누락값 개수 구함.
    return null_count

In [33]:
cmis_col = titanic.apply(count_missing)
print(cmis_col)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


In [35]:
#누락값 비중을 확인하는 함수 생성: prop_missing
def prop_missing(vec):
    num = count_missing(vec)   #결측치 개수
    dem = vec.size             #데이터 프레임의 전체 데이터 수
    return num / dem

In [36]:
pmis_col = titanic.apply(prop_missing)
print(pmis_col)

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64


In [37]:
#누락값을 제외한 데이터의 비중 구하기.
def prop_complete(vec):
    return 1 - prop_missing(vec)

In [38]:
#결측치 처리하기(행 방향)
cmis_row = titanic.apply(count_missing, axis = 1)
pmis_row = titanic.apply(prop_missing, axis = 1)
pcom_row = titanic.apply(prop_complete, axis = 1)

print(cmis_row.head())
print(pmis_row.head())
print(pcom_row.head())

0    1
1    0
2    1
3    0
4    1
dtype: int64
0    0.066667
1    0.000000
2    0.066667
3    0.000000
4    0.066667
dtype: float64
0    0.933333
1    1.000000
2    0.933333
3    1.000000
4    0.933333
dtype: float64


In [40]:
#결측치 개수를 구해 titanic 데이터프레임에 추가하기.
titanic['num_missing'] = titanic.apply(count_missing, axis = 1)
print(titanic.head())

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  num_missing  
0    man        True  NaN  Southampton    no  False            1  
1  woman       False    C    Cherbourg   yes  False            0  
2  woman       False  NaN  Southampton   yes   True            1  
3  woman       False    C  Southampton   yes  False            0  
4    man        True  NaN  Southampton    no   True            1  


In [44]:
#결측치가 있는 데이터만 따로 모아서 볼 수도 있음.
print(titanic.loc[titanic.num_missing > 1, :].sample(10)) #결측치가 2개 이상인 데이터를 무작위로 10개 추출

     survived  pclass     sex  age  sibsp  parch      fare embarked  class  \
656         0       3    male  NaN      0      0    7.8958        S  Third   
180         0       3  female  NaN      8      2   69.5500        S  Third   
48          0       3    male  NaN      2      0   21.6792        C  Third   
28          1       3  female  NaN      0      0    7.8792        Q  Third   
573         1       3  female  NaN      0      0    7.7500        Q  Third   
667         0       3    male  NaN      0      0    7.7750        S  Third   
650         0       3    male  NaN      0      0    7.8958        S  Third   
828         1       3    male  NaN      0      0    7.7500        Q  Third   
334         1       1  female  NaN      1      0  133.6500        S  First   
330         1       3  female  NaN      2      0   23.2500        Q  Third   

       who  adult_male deck  embark_town alive  alone  num_missing  
656    man        True  NaN  Southampton    no   True            2  
180